<CENTER>
    <a href="http://opendata.atlas.cern" class="icons"><img src="../Assets/ATLASOD.gif" style="width:50%"></a>
</CENTER>

# A more difficult notebook in python

### In this notebook you can find a more difficult program that shows further high energy physics (HEP) analysis techniques.

##### The following analysis is searching for events where [Z bosons](https://en.wikipedia.org/wiki/W_and_Z_bosons) decay to two leptons of same flavour and opposite charge (to be seen for example in the [Feynman diagram](https://en.wikipedia.org/wiki/Feynman_diagram)).

<CENTER><img src="../Assets/Z_ElectronPositron.png" style="width:30%"></CENTER>

First of all - like we did it in the first notebook - ROOT is imported to read the files in the _.root_ data format.

In [3]:
import ROOT

Welcome to JupyROOT 6.18/04


In order to activate the interactive visualisation of the histogram that is later created we can use the JSROOT magic:

In [4]:
%jsroot on

Next we have to open the data that we want to analyze. As described above the data is stored in a _*.root_ file.

In [5]:
!wget http://opendata.cern.ch/eos/opendata/atlas/OutreachDatasets/2020-01-22/4lep/MC/mc_307431.RS_G_ZZ_llll_c10_m0200.4lep.root

--2020-07-23 15:15:22--  http://opendata.cern.ch/eos/opendata/atlas/OutreachDatasets/2020-01-22/4lep/MC/mc_307431.RS_G_ZZ_llll_c10_m0200.4lep.root
Resolving opendata.cern.ch (opendata.cern.ch)... failed: Name or service not known.
wget: unable to resolve host address ‘opendata.cern.ch’


In [6]:
!ls -l
!pwd


total 19528
-rw-r--r-- 1 student student   28504 Jan 28 23:12 ATLAS_OpenData_01-cpp_Hyy_channel_analysis_example.ipynb
-rw-r--r-- 1 student student   27309 Jan 28 23:10 ATLAS_OpenData_01-python_Hyy_channel_analysis_example.ipynb
-rw-r--r-- 1 student student   15274 Jan 28 01:23 ATLAS_OpenData_01-simple_cpp_example_histogram.ipynb
-rw-r--r-- 1 student student   24269 Mär 12 16:59 ATLAS_OpenData_02-simple_python_example_histogram.ipynb
-rw-r--r-- 1 student student   16924 Jan 28 01:45 ATLAS_OpenData_03-cpp_invariant_mass_reconstruction_using_TLorentz_vectors.ipynb
-rw-r--r-- 1 student student   24824 Jul 23 15:14 ATLAS_OpenData_04-python_invariant_mass_reconstruction_using_TLorentz_vectors.ipynb
-rw-r--r-- 1 student student 3389717 Jan 28 01:49 ATLAS_OpenData_05-ccp_CSV_to_ROOT_ploting_example.ipynb
-rw-r--r-- 1 student student   13694 Jan 28 01:50 ATLAS_OpenData_06-cpp_simple_cut_and_count_analysis_example.ipynb
-rw-r--r-- 1 student student   38206 Mär 24 20:30 ATLAS_OpenData_07-python_

In [7]:
## Example of local file ##
## f = ROOT.TFile.Open("/home/student/datasets/MC/mc_105986.ZZ.root")
## Example of remote file (URL) ##

f = ROOT.TFile.Open("mc_307431.RS_G_ZZ_llll_c10_m0200.4lep.root")
## f = ROOT.TFile.Open("http://opendata.atlas.cern/release/samples/MC/mc_147770.Zee.root")

After the data is opened we create a canvas on which we can draw a histogram. If we do not have a canvas we cannot see our histogram at the end. Its name is _Canvas_ and its header is _c_. The two following arguments define the width and the height of the canvas.

In [21]:
canvas = ROOT.TCanvas("Canvas","c",800,600)

Warning in <TCanvas::Constructor>: Deleting canvas with same name: Canvas


The next step is to define a tree named _t_ to get the data out of the _.root_ file.

In [9]:
tree = f.Get("mini")
tree.GetEntries()




4333

Now we define a histogram that will later be placed on this canvas. Its name is _variable_, the header of the histogram is _Mass of the Z boson_, the x axis is named _mass [GeV]_ and the y axis is named _events_. The three following arguments indicate that this histogram contains 30 bins which have a range from 40 to 140.

In [10]:
hist = ROOT.TH1F("variable","Mass of the Z boson; mass [GeV]; events",30,40,140)

Time to fill our above defined histogram. At first we define some variables and then we loop over the data. We also make some cuts as you can see in the # _comments_.

In [11]:
def iterations(number_i):
    counter = 0
    sets = []
    while counter < number_i:
        counter2 = counter
        while counter2 < number_i:
            iteration = []
            if counter == counter2:
                counter2 = counter2 + 1
            iteration.append(counter)
            iteration.append(counter2)
            counter2 = counter2 + 1
            sets.append(iteration)
        counter = counter +1
    sets.pop()
    return sets 

leadLepton  = ROOT.TLorentzVector()
trailLepton = ROOT.TLorentzVector()
combinations4 = iterations(4)
def fndMass2lep(x, y):
    if tree.lep_charge[x] != tree.lep_charge[y]:
            
                    if (tree.lep_type[x] == tree.lep_type[y]):

                        # Let's define one TLorentz vector for each, e.i. two vectors!
                        leadLepton.SetPtEtaPhiE(tree.lep_pt[x]/1000., tree.lep_eta[x], tree.lep_phi[x], tree.lep_E[x]/1000.)
                        trailLepton.SetPtEtaPhiE(tree.lep_pt[y]/1000., tree.lep_eta[y], tree.lep_phi[y], tree.lep_E[y]/1000.)
                        # Next line: addition of two TLorentz vectors above --> ask mass very easy (devide by 1000 to get value in GeV)
                        invmass = leadLepton + trailLepton
                        return invmass
                    


In [12]:
def findMissing(x,y):
    numbers = [0,1,2,3]
    numbers.remove(x)
    numbers.remove(y)
    return numbers
print(findMissing(3,2))

[0, 1]


In [18]:



for event in tree:
    if tree.lep_n >= 4:
        problem = False
        n_electrons = 0
        for lepton in tree.lep_type:
            if lepton == 11:
                n_electrons = n_electrons + 1
        if tree.lep_n == 4:
            if n_electrons == 0 or n_electrons == 4:
                problem = True
        if tree.lep_n > 4:
            problem == True
            
        if problem:
            if tree.lep_n == 4:
                counter_combination = 0
                lepscom = []
                lepscom_is_something = False
                for combination in combinations4:
                    if counter_combination < 3:
                        x = combination[0]
                        y = combination[1]
                        invmass1 = fndMass2lep(x, y)
                        counter_combination = counter_combination + 1
                        if invmass1 is not None:
                            other = findMissing(x, y)
                            x2 = other[0]
                            y2 = other[1]
                            invmass2 = fndMass2lep(x, y)
                            deviation = (90-invmass2.M()) * (90-invmass2.M()) + (90-invmass1.M()) * (90-invmass1.M())
                            lepcom = {}
                            lepcom["invmasscom"] = [invmass2.M() , invmass1.M()]
                            lepcom["deviation"] = deviation
                            lepscom.append(lepcom)
                        counter_combination = counter_combination + 1
                if len(lepscom) == 2:
                    if lepscom[0]["deviation"] < lepscom[1]["deviation"]:
                        for invmass in lepscom[0]["invmasscom"]:
                            hist.Fill(invmass)
                    else:
                        for invmass in lepscom[1]["invmasscom"]:
                            hist.Fill(invmass)

           
                        
                        

        if problem is not True:
            for combination in combinations4:
                x = combination[0]
                y = combination[1]
                invmass = fndMass2lep(x, y)
                if invmass != None:
                    hist.Fill(invmass.M())


        
                
        
        
        
            

In [41]:
p = iterations(4)
print(p)

[[0, 1], [0, 2], [0, 3], [1, 2], [1, 3], [2, 3]]


After filling the histogram we want to see the results of the analysis. First we draw the histogram on the canvas and then the canvas on which the histogram lies.

In [19]:
hist.Draw()


In [20]:
canvas.Draw()

**Done!**